# A Scrap-reducing Scheduling Strategy for Batch Production

A spreadsheet containing one year of operational data indicates several reasons for scrap. In this notebook, we assume that the following values listed in the column **scrap reasons** indicate pigment from the prior batch contaminating the next batch, creating the scrap:
* Tile Shade
* Tile Quality
* Offshade
* Foreign Color
* Patt. Change

These scrap reasons account for 15.6% of total scrap. 

We model the scheduling problem as a [Traveling Salesman Problem](https://en.wikipedia.org/wiki/Travelling_salesman_problem). Where replacing "distance between cities" is one of:

1. an empirical model of "scrap between product types" or, 
2. an analytical model describing the difference between color as it affects scrap, or
3. some hybrid combination of (1) and (2). 

To use the empirical approach, ideally we would have accurate historical operational data indicating the scrap produced between every pair of products. Of course, we don't have that. 

## Do it in MiniZinc

In [ ]:
%%minizinc -m bind

include "globals.mzn";

int n;
array [1..n,1..n] of int: dist;
int: start_city;
int end_city;

array[1..n] of var 1..n city;
array[1..n] of string: city_name;
    
constraint city[1] = start_city;
constraint city[n] = end_city;

int total_distance = sum(i in 2..n)(dist[city[i-1],city[i]]);
solve minimize total_distance;

## Never Stop Improving!

![alt text](https://raw.githubusercontent.com/pdenno/minizinc-notebooks/master/images/jupyter-files-view.png "Logo Title Text 1")